In [40]:
import pandas as pd

import os
from openai import OpenAI

from concurrent.futures import ThreadPoolExecutor
from itertools import repeat

In [ ]:
# 腾讯

import os
from openai import OpenAI

# 构造 client
client = OpenAI(
    api_key='sk-O7xIcdGrTqEwYmhqfPRsEO2iNDY6DdQbkYDzgQpAPVXN8TOJ', # 混元 APIKey
    base_url="https://api.hunyuan.cloud.tencent.com/v1", # 混元 endpoint
)

# 自定义参数传参示例
completion = client.chat.completions.create(
    model="hunyuan-turbo",
    messages=[
        {
            "role": "user",
            "content": prompt,
        },
    ],
    extra_body={
        "enable_enhancement": True, # <- 自定义参数
    },
)

In [5]:
df_main_host = pd.read_excel("../data/主域名待分类.xlsx")
df_main_host.head()

,主域名,备案主体,网站名称,网站首页网址
0,peopledailyhealth.com,《健康时报》社有限责任公司,健康客户端,www.peopledailyhealth.com
1,coloros.com,OPPO广东移动通信有限公司,ColorOS官方网站,www.coloros.com
2,alikunlun.com,阿里巴巴（中国）有限公司,阿里云内部使用,www.alikunlun.com
3,gtm-a5b2.com,阿里云长三角（嘉善）信息科技有限公司,阿里云DNS流量调度平台,www.gtm-a5b2.com
4,zepp.com,安徽华米信息科技有限公司,安徽华米zepp官网,www.zepp.com


In [7]:
df_main_host['content'] = df_main_host['主域名'].str.cat(df_main_host['网站名称'], sep='|')

In [8]:
df_main_host

,主域名,备案主体,网站名称,网站首页网址,content
0,peopledailyhealth.com,《健康时报》社有限责任公司,健康客户端,www.peopledailyhealth.com,peopledailyhealth.com|健康客户端
1,coloros.com,OPPO广东移动通信有限公司,ColorOS官方网站,www.coloros.com,coloros.com|ColorOS官方网站
2,alikunlun.com,阿里巴巴（中国）有限公司,阿里云内部使用,www.alikunlun.com,alikunlun.com|阿里云内部使用
3,gtm-a5b2.com,阿里云长三角（嘉善）信息科技有限公司,阿里云DNS流量调度平台,www.gtm-a5b2.com,gtm-a5b2.com|阿里云DNS流量调度平台
4,zepp.com,安徽华米信息科技有限公司,安徽华米zepp官网,www.zepp.com,zepp.com|安徽华米zepp官网
...,...,...,...,...,...
1765,flyme.cn,珠海市魅族通讯设备有限公司,flyme官网,www.flyme.cn,flyme.cn|flyme官网
1766,mzres.com,珠海市魅族通讯设备有限公司,Flyme系统更新,www.mzres.com,mzres.com|Flyme系统更新
1767,bizport.cn,珠海市小源科技有限公司,珠海市小源科技短信公众号,www.bizport.cn,bizport.cn|珠海市小源科技短信公众号
1768,meizu.com,珠海星纪魅族信息技术有限公司,魅族官网,www.meizu.com,meizu.com|魅族官网


In [9]:
df_feature = pd.read_table('../data/feature.txt', header=None)
df_feature.columns = ['feature']
cat_str = '|'.join(list(df_feature['feature']))
cat_str

'保险|金融理财|社交|婚恋相亲|生活服务|影音娱乐|新闻阅读|游戏|汽车|教育学习|电商购物|医疗健康|丽人图像|科教文化服务|母婴亲子|旅游出行'

In [31]:
gpt_content = dict()

In [23]:
# 构造 client
client = OpenAI(
    api_key='sk-O7xIcdGrTqEwYmhqfPRsEO2iNDY6DdQbkYDzgQpAPVXN8TOJ', # 混元 APIKey
    base_url="https://api.hunyuan.cloud.tencent.com/v1", # 混元 endpoint
)

In [49]:
def fetch_response(cat_str, host_content):
    print(host_content)
    prompt = f"你是一个域名分类器，你的任务是根据主域名、网站名称对域名进行分类，分类结果有:{cat_str}，请对{host_content} 进行分类"
    completion = client.chat.completions.create(
        model="hunyuan-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
        extra_body={
            "enable_enhancement": True,
        },
    )
    gpt_content[host_content] = completion.choices[0].message.content

In [61]:
with ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(fetch_response, repeat(cat_str), df_main_host['content'])

myjiedian.com|节点科技
lkbaofu.com|兰科优品
jetour.com.cn|捷途汽车官网
37.com.cn|37手游网
nm-pay.cn|安徽野象
r2j.cn|安徽云涌信息技术
dl02.cn|掌榕网短链站
zhixue.com|智学网
qmai.cn|企迈云商
mye3.cn|蜂鸟门户
vwg-connect.cn|大众互联驾驶
baihe.com|百合网
bestv.com.cn|百视通官网
babybus.com|宝宝巴士
weshineapp.com|北京艾享科技有限责任公司
qiyi.com|爱奇艺
iqiyi.com|爱奇艺
bsgslb.cn|白山云科技
duxiaomanfund.com|北京度小满基金销售有限公司
bcebos.com|百度公有云-开放云
bdstatic.com|百度网页检索静态内容服务
bdimg.com|北京百度网讯科技有限公司
shifen.com|百度
bdycdn.cn|百度云CDN
jomodns.com|JOMODNS
bcevod.com|北京百度网讯科技有限公司
bdydns.com|百度云DNS
baifubao.com|百付宝
smartapps.cn|智能小程序门户
qznovelpic.com|红果官网
biduwenxue.com|北京必读科技有限公司
jxbrty.com|北京博瑞彤芸科技股份有限公司
caiyunapp.com|彩云科技
ifreetalk.com|北京畅聊天下科技股份有限公司
chaonei.com|北京巢内科技发展有限公司
dailygn.com|朝夕光年官网-激发创造，丰富生活
bdgp.cc|北京朝夕光年信息技术有限公司
autohome.com.cn|汽车之家
autoimg.cn|汽车之家图片网
athm.cn|汽车之家-athm
chenzhongkj.cn|晨钟科技
chenzhongkj.com|晨钟科技
adwery.com|诚联网
cs30.net|Elva客服
aihelp.net|AIHELP智能系统
tobidad.cn|北京创致科技有限公司官网
bidtoolads.com|北京创致科技有限公司
sigmob.cn|北京创智汇聚科技股份有限公司
doubao.com|豆包
chunyuyisheng.com|春雨医生
chu

In [62]:
gpt_content

{'coloros.com|ColorOS官方网站': '根据提供的域名和网站名称信息，“coloros.com”和“ColorOS官方网站”应该归类为“科教文化服务”。这是因为ColorOS是OPPO推出的操作系统，其官方网站主要提供关于该操作系统的介绍、使用教程、技术支持等科教文化服务相关的内容。\n\n因此，分类结果为：科教文化服务',
 'peopledailyhealth.com|健康客户端': '根据提供的域名和网站名称信息，我将对其进行分类如下：\n\n**域名:** peopledailyhealth.com  \n**网站名称:** 健康客户端\n\n**分类:** 医疗健康\n\n**理由:** 域名中的“health”（健康）以及网站名称“健康客户端”都明确指向了医疗健康领域，因此该域名最适合被分类到“医疗健康”这一类别。',
 'zepp.com|安徽华米zepp官网': '根据提供的域名信息，zepp.com 和 安徽华米zepp官网 可以归类为“科教文化服务”。以下是具体分析：\n\n### zepp.com\n- **主域名**：zepp\n- **网站名称**：无（仅域名）\n- **分类理由**：Zepp 是一个知名的健康与健身追踪品牌，提供智能手环和相关健康数据服务。虽然其产品涉及健康监测，但其核心业务更多是围绕科技和数据服务展开，因此更适合归类到“科教文化服务”类别中。\n\n### 安徽华米zepp官网\n- **主域名**：安徽华米zepp官网\n- **网站名称**：安徽华米zepp官网\n- **分类理由**：安徽华米是Zepp品牌在中国的一个重要合作伙伴，主要负责生产和销售相关智能穿戴设备。官网主要用于展示产品和提供相关信息，依然与科技和数据服务相关，因此也归类为“科教文化服务”。\n\n### 总结\n- **zepp.com**：科教文化服务\n- **安徽华米zepp官网**：科教文化服务\n\n这种分类方式更侧重于域名所代表的业务性质和技术属性。如果有更具体的分类需求，可以考虑细化到“科技产品”或“健康管理”等子类别。',
 'gtm-a5b2.com|阿里云DNS流量调度平台': '根据提供的域名“gtm-a5b2.com”和描述“阿里云DNS流量调度平台”，这个域名和描述更倾向于技术或服务类的用途，而不是直

In [63]:
df_cat = pd.DataFrame(gpt_content.items(), columns=['content', 'result'])
df_cat.to_excel("../data/主域名分类detail.xlsx", index=False)

In [64]:
df_cat.count()

content    1770
result     1770
dtype: int64

In [68]:
df_cat_detail = pd.read_excel("../data/主域名分类detail.xlsx")
df_cat_detail.count()

content    2062
主域名        2062
网站名称       2062
result     2062
dtype: int64

In [74]:
df_cat_detail.head()

,content,主域名,网站名称,result
0,coloros.com|ColorOS官方网站,coloros.com,ColorOS官方网站,根据提供的域名和网站名称信息，“coloros.com”和“ColorOS官方网站”应该归类...
1,peopledailyhealth.com|健康客户端,peopledailyhealth.com,健康客户端,根据提供的域名和网站名称信息，我将对其进行分类如下：\n\n**域名:** peopleda...
2,zepp.com|安徽华米zepp官网,zepp.com,安徽华米zepp官网,根据提供的域名信息，zepp.com 和 安徽华米zepp官网 可以归类为“科教文化服务”。...
3,gtm-a5b2.com|阿里云DNS流量调度平台,gtm-a5b2.com,阿里云DNS流量调度平台,根据提供的域名“gtm-a5b2.com”和描述“阿里云DNS流量调度平台”，这个域名和描述...
4,alikunlun.com|阿里云内部使用,alikunlun.com,阿里云内部使用,"根据提供的信息，""alikunlun.com"" 和 ""阿里云内部使用"" 这个域名看起来是与阿..."


In [73]:
df_cat_detail['result'].str.extractall(f'({"|".join(df_feature["feature"])})')[0].groupby(level=0).apply('|'.join)

0                     科教文化服务|科教文化服务|科教文化服务
1                           医疗健康|医疗健康|医疗健康
2       科教文化服务|科教文化服务|科教文化服务|科教文化服务|科教文化服务
3       金融理财|电商购物|金融理财|金融理财|金融理财|金融理财|金融理财
4            金融理财|金融理财|金融理财|金融理财|金融理财|金融理财
                       ...                
2057                         科教文化服务|科教文化服务
2058                        生活服务|生活服务|生活服务
2059                                  金融理财
2060                                  新闻阅读
2061                   电商购物|旅游出行|旅游出行|旅游出行
Name: 0, Length: 2042, dtype: object

In [75]:
# 提取result列的文字
results = df_cat_detail['result'].str.extractall(f'({"|".join(df_feature["feature"])})')[0].groupby(level=0).apply('|'.join)
df_cat_detail['category'] = results
df_cat_detail['category'] = df_cat_detail['category'].apply(lambda x: '|'.join(sorted(set(str(x).split('|')))) if isinstance(x, str) else '')

In [77]:
df_cat_detail.sample(10)

,content,主域名,网站名称,result,category
1141,redbeeai.com|沙猫AI先锋,redbeeai.com,沙猫AI先锋,"根据提供的域名和网站名称信息，我会将 ""redbeeai.com|沙猫AI先锋"" 进行如下分...",科教文化服务
738,rrxiu.net|人人秀,rrxiu.net,人人秀,根据提供的域名和网站名称信息，“rrxiu.net|人人秀”可以归类为“社交”。\n\n理由...,社交
186,cmcm.com|猎豹移动官网,cmcm.com,猎豹移动官网,cmcm.com 和 猎豹移动官网 这两个域名可以归类为“辅助服务”或“其他”，但在给出的分...,游戏
859,wosms.cn|联通在线卡部消息能力,wosms.cn,联通在线卡部消息能力,根据提供的域名“wosms.cn”和描述“联通在线卡部消息能力”，这个域名和描述更倾向于与通...,社交
892,blackshark.com|南昌黑鲨科技官网,blackshark.com,南昌黑鲨科技官网,根据提供的域名信息，blackshark.com 和 南昌黑鲨科技官网 可以分类为**游戏*...,游戏
1823,baigebao.com|白鸽宝,baigebao.com,白鸽宝,根据提供的域名“baigebao.com”和网站名称“白鸽宝”，这个域名很可能与金融服务相关...,金融理财
269,ydstatic.com|网易有道,ydstatic.com,网易有道,"根据提供的信息，""ydstatic.com"" 和 ""网易有道"" 的分类如下：\n\n1. y...",教育学习|科教文化服务
1237,zylauncher.com|上海卓悠网络科技有限公司,zylauncher.com,上海卓悠网络科技有限公司,根据提供的域名“zylauncher.com”和网站名称“上海卓悠网络科技有限公司”，可以进...,电商购物
1286,myaqsh.com|公司APP下载网站,myaqsh.com,公司APP下载网站,"根据提供的信息，""myaqsh.com"" 和 ""公司APP下载网站"" 这两个域名可以进行如下...",生活服务
812,weilianmenggz.cn|嘉兴复力文化传媒有限公司,weilianmenggz.cn,嘉兴复力文化传媒有限公司,根据提供的域名和公司名称信息，可以进行如下分类：\n\n**weilianmenggz.cn...,科教文化服务


In [81]:
df_cat_detail.to_excel("../data/主域名分类备份.xlsx", index=False)